In [13]:
%env KERAS_BACKEND = tensorflow
%matplotlib inline

# Sequential 不能實做 branch,merge 網路結構, 使用 model API

import numpy as np
import matplotlib.pyplot as plt

# prepare dataset for training and testing

from keras.datasets import mnist
from keras.utils import np_utils

(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train = x_train.reshape(60000,784)
x_test =x_test.reshape(10000,784)

y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

print('datas : ',x_train.shape,x_test.shape,y_train.shape,y_test.shape)

from keras.models import Model
from keras.optimizers import SGD
from keras.layers import Input,Dense,Activation

from keras.layers import concatenate,add

# 784 -(f1)-> 500 -(f2)-> 500 -(f3)-> 10
#                ↘ (f4)-> 500 -(f3)-↗

f_1 = Dense(500,activation ='sigmoid')
f_2 = Dense(500,activation='sigmoid')
f_3 = Dense(10,activation = 'softmax')

print('f_1 : ',f_1)

# 第一層才需定義 input 的 dimension

x = Input(shape=(784,))

print('x : ',x)

h_1 = f_1(x)
h_2 = f_2(h_1)
y = f_3(h_2)

print('h_1 : ',h_1)
print('h_2 : ',h_2)
print('y : ',y)

# branch structure
f_4 = Dense(500,activation='relu')
z=f_4(h_1)

#new f_3 for merge structure
f_3 = Dense(10,activation='softmax')

'''
error :
y = f_3(h_2,z)
y = f_3([h_2,z])

'''
#correct : 

u = concatenate([h_2,z])
y = f_3(u)

print('u : ',u)
print('y : ',y)

# result of concatenate : see 'connected to' im summary
# 784 -(f1)-> 500 -(f2)-> 500 --> 1000(u) -(f3)-> 10(y)
#                ↘ (f4)-> 500 --↗

model = Model(x,y)
model.summary()

# 常用應用
# CNN - U-net mutil-resolution fusion(MRF)
# ResNet - skip connection, 但ResNet 使用 add 不是 concatenate

env: KERAS_BACKEND=tensorflow
datas :  (60000, 784) (10000, 784) (60000, 10) (10000, 10)
f_1 :  <keras.layers.core.Dense object at 0x000002A503E74C18>
x :  Tensor("input_10:0", shape=(?, 784), dtype=float32)
h_1 :  Tensor("dense_41/Sigmoid:0", shape=(?, 500), dtype=float32)
h_2 :  Tensor("dense_42/Sigmoid:0", shape=(?, 500), dtype=float32)
y :  Tensor("dense_43/Softmax:0", shape=(?, 10), dtype=float32)
u :  Tensor("concatenate_5/concat:0", shape=(?, 1000), dtype=float32)
y :  Tensor("dense_45/Softmax:0", shape=(?, 10), dtype=float32)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 784)          0                                            
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 500)    